# Notebook 04 — Operational Thresholding under Capacity Constraints

This notebook translates model probabilities into actionable decisions.

Rather than optimising ROC-AUC alone, we simulate a realistic constraint:
the performance staff can only review a fixed percentage of players per match cycle.


## 1. Setup


In [11]:
import os
import numpy as np
import pandas as pd
import duckdb

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score

DB_PATH = os.getenv("FRA_DUCKDB_PATH", "../lakehouse/analytics.duckdb")
TABLE = "player_dataset_predictive_v2"

FEATURES = [
    "minutes_last_7d","minutes_last_14d","minutes_last_28d","minutes_last_5_matches","acwr",
    "minutes_std_last_5_matches","minutes_std_last_10_matches",
    "delta_7d_14d","delta_14d_28d",
    "ratio_7d_14d","ratio_14d_28d",
    "acwr_change",
    "season_minutes_cum","season_matches_played","season_avg_minutes",
    "minutes_last_3_matches","season_momentum_3v_season_avg"
]
TARGET = "high_risk_next"


## 2. Load Data (chronological split)


In [8]:
con = duckdb.connect(DB_PATH, read_only=True)
df = con.execute(f"SELECT * FROM {TABLE}").fetchdf()
con.close()

df = df.sort_values("match_date").reset_index(drop=True)

split = int(len(df)*0.8)
train = df.iloc[:split]
test = df.iloc[split:]

X_train, y_train = train[FEATURES], train[TARGET]
X_test, y_test = test[FEATURES], test[TARGET]


## 3. Model Training (Logistic Regression)


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

model = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=2000))
])

model.fit(X_train, y_train)
p_train = model.predict_proba(X_train)[:, 1]
p_test  = model.predict_proba(X_test)[:, 1]


## 4. Capacity-Constrained Thresholding


In [12]:
def threshold_for_capacity(p_train, capacity):
    return float(np.quantile(p_train, 1 - capacity))

def eval_policy(y_true, p, thr):
    pred = (p >= thr).astype(int)
    return {
        "precision": precision_score(y_true, pred, zero_division=0),
        "recall": recall_score(y_true, pred, zero_division=0),
        "f1": f1_score(y_true, pred, zero_division=0),
        "alert_rate": pred.mean(),
        "alerts_per_25": pred.mean() * 25
    }

results = []
for cap in [0.05, 0.10, 0.20]:
    thr = threshold_for_capacity(p_train, cap)

    tr = eval_policy(y_train, p_train, thr)
    te = eval_policy(y_test,  p_test,  thr)

    results.append({
        "capacity": cap,
        "threshold": thr,
        "train_alert_rate": tr["alert_rate"],
        "test_alert_rate": te["alert_rate"],
        "test_precision": te["precision"],
        "test_recall": te["recall"],
        "test_f1": te["f1"],
        "expected_alerts_per_25": te["alerts_per_25"],
    })

pd.DataFrame(results)


,capacity,threshold,train_alert_rate,test_alert_rate,test_precision,test_recall,test_f1,expected_alerts_per_25
0,0.05,0.640352,0.050011,0.044357,0.380556,0.035691,0.065261,1.108921
1,0.10,0.609941,0.100006,0.067521,0.402372,0.057444,0.100536,1.688024
2,0.20,0.563698,0.200798,0.121119,0.457782,0.117233,0.186664,3.027969


## 5. Operational Insight

Higher capacity increases recall but reduces precision.
The optimal capacity depends on staff bandwidth and tolerance for false positives.

Thresholding is an organisational decision, not purely statistical.



# 6. Final Conclusions — Operational Decision Layer

This notebook demonstrates how predictive probabilities are translated into actionable decisions under real-world constraints.

Key takeaways:

- The model shows coherent ranking behaviour across risk levels.
- Increasing monitoring capacity significantly improves recall.
- Precision remains relatively stable, indicating structural signal.
- The optimal threshold is not purely statistical — it is organisational.

From a performance analytics perspective:

- 5% capacity is highly conservative.
- 10% capacity provides a balanced monitoring shortlist.
- 20% capacity maximises detection but increases workload demand.

This notebook formalises the decision layer between model output and staff action.

## Operational Interpretation Under Capacity Constraints

Thresholds were selected using training quantiles to simulate
real-world medical or performance staff review capacity.

Key insights:

- At 10% alert capacity:
  - Precision ≈ 0.75
  - Recall ≈ 0.15

This indicates that the model is conservative:
it prioritizes high-confidence risk flags over broad recall.

Such behaviour may be desirable in professional environments
where false positives carry operational cost.

Threshold selection must align with organizational tolerance
for missed risk vs review workload.